# SGLang Native APIs

Apart from the OpenAI compatible APIs, the SGLang Runtime also provides its native server APIs. We introduce these following APIs:

- `/generate` (text generation model)
- `/get_model_info`
- `/get_server_info`
- `/health`
- `/health_generate`
- `/flush_cache`
- `/update_weights`
- `/encode`(embedding model)
- `/v1/rerank`(cross encoder rerank model)
- `/classify`(reward model)
- `/start_expert_distribution_record`
- `/stop_expert_distribution_record`
- `/dump_expert_distribution_record`

We mainly use `requests` to test these APIs in the following examples. You can also use `curl`.

## Launch A Server

In [1]:
import requests
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process


server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b-instruct --host 0.0.0.0"
)
## To run qwen2.5-0.5b-instruct model on the Ascend-Npu, you can execute the following command:
# server_process, port = launch_server_cmd(
#     "python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b-instruct --host 0.0.0.0 --device npu --tp 2 --attention-backend torch_native"
# )

wait_for_server(f"http://localhost:{port}")

[2025-08-03 10:09:26] server_args=ServerArgs(model_path='qwen/qwen2.5-0.5b-instruct', tokenizer_path='qwen/qwen2.5-0.5b-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=39756, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.874, max_running_requests=200, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=619767944, co

[2025-08-03 10:09:27] Using default HuggingFace chat template with detected content format: string


[2025-08-03 10:09:40] Attention backend not explicitly specified. Use fa3 backend by default.
[2025-08-03 10:09:40] Init torch distributed begin.


[2025-08-03 10:09:42] Init torch distributed ends. mem usage=0.00 GB


[2025-08-03 10:09:44] Load weight begin. avail mem=75.57 GB


...

[2025-08-03 10:09:47] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=46.47 GB


[2025-08-03 10:09:47] INFO:     Started server process [3923617]
[2025-08-03 10:09:47] INFO:     Waiting for application startup.
[2025-08-03 10:09:47] INFO:     Application startup complete.
[2025-08-03 10:09:47] INFO:     Uvicorn running on http://0.0.0.0:39756 (Press CTRL+C to quit)


[2025-08-03 10:09:48] INFO:     127.0.0.1:54580 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-03 10:09:48] INFO:     127.0.0.1:54586 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-03 10:09:48] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-03 10:09:50] INFO:     127.0.0.1:54590 - "POST /generate HTTP/1.1" 200 OK
[2025-08-03 10:09:50] The server is fired up and ready to roll!


## Generate (text generation model)
Generate completions. This is similar to the `/v1/completions` in OpenAI API. Detailed parameters can be found in the [sampling parameters](./sampling_params.md).

In [2]:
url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-08-03 10:09:53] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-03 10:09:53] Decode batch. #running-req: 1, #token: 40, token usage: 0.00, cuda graph: False, gen throughput (token/s): 6.21, #queue-req: 0, 


[2025-08-03 10:09:53] Decode batch. #running-req: 1, #token: 80, token usage: 0.00, cuda graph: False, gen throughput (token/s): 128.83, #queue-req: 0, 


[2025-08-03 10:09:54] Decode batch. #running-req: 1, #token: 120, token usage: 0.01, cuda graph: False, gen throughput (token/s): 127.97, #queue-req: 0, 
[2025-08-03 10:09:54] INFO:     127.0.0.1:54604 - "POST /generate HTTP/1.1" 200 OK


## Get Model Info

Get the information of the model.

- `model_path`: The path/name of the model.
- `is_generation`: Whether the model is used as generation model or embedding model.
- `tokenizer_path`: The path/name of the tokenizer.
- `preferred_sampling_params`: The default sampling params specified via `--preferred-sampling-params`. `None` is returned in this example as we did not explicitly configure it in server args.

In [3]:
url = f"http://localhost:{port}/get_model_info"

response = requests.get(url)
response_json = response.json()
print_highlight(response_json)
assert response_json["model_path"] == "qwen/qwen2.5-0.5b-instruct"
assert response_json["is_generation"] is True
assert response_json["tokenizer_path"] == "qwen/qwen2.5-0.5b-instruct"
assert response_json["preferred_sampling_params"] is None
assert response_json.keys() == {
    "model_path",
    "is_generation",
    "tokenizer_path",
    "preferred_sampling_params",
}

[2025-08-03 10:09:54] INFO:     127.0.0.1:54610 - "GET /get_model_info HTTP/1.1" 200 OK


## Get Server Info
Gets the server information including CLI arguments, token limits, and memory pool sizes.
- Note: `get_server_info` merges the following deprecated endpoints:
  - `get_server_args`
  - `get_memory_pool_size` 
  - `get_max_total_num_tokens`

In [4]:
# get_server_info

url = f"http://localhost:{port}/get_server_info"

response = requests.get(url)
print_highlight(response.text)

[2025-08-03 10:09:54] INFO:     127.0.0.1:54622 - "GET /get_server_info HTTP/1.1" 200 OK


## Health Check
- `/health`: Check the health of the server.
- `/health_generate`: Check the health of the server by generating one token.

In [5]:
url = f"http://localhost:{port}/health_generate"

response = requests.get(url)
print_highlight(response.text)

[2025-08-03 10:09:54] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-03 10:09:55] INFO:     127.0.0.1:54630 - "GET /health_generate HTTP/1.1" 200 OK


In [6]:
url = f"http://localhost:{port}/health"

response = requests.get(url)
print_highlight(response.text)

[2025-08-03 10:09:55] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-03 10:09:56] INFO:     127.0.0.1:54632 - "GET /health HTTP/1.1" 200 OK


## Flush Cache

Flush the radix cache. It will be automatically triggered when the model weights are updated by the `/update_weights` API.

In [7]:
# flush cache

url = f"http://localhost:{port}/flush_cache"

response = requests.post(url)
print_highlight(response.text)

[2025-08-03 10:09:56] Cache flushed successfully!
[2025-08-03 10:09:56] INFO:     127.0.0.1:42108 - "POST /flush_cache HTTP/1.1" 200 OK


## Update Weights From Disk

Update model weights from disk without restarting the server. Only applicable for models with the same architecture and parameter size.

SGLang support `update_weights_from_disk` API for continuous evaluation during training (save checkpoint to disk and update weights from disk).


In [8]:
# successful update with same architecture and size

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "qwen/qwen2.5-0.5b-instruct"}

response = requests.post(url, json=data)
print_highlight(response.text)
assert response.json()["success"] is True
assert response.json()["message"] == "Succeeded to update model weights."

[2025-08-03 10:09:56] Start update_weights. Load format=auto
[2025-08-03 10:09:56] Update engine weights online from disk begin. avail mem=41.50 GB
[2025-08-03 10:09:56] Using model weights format ['*.safetensors']
[2025-08-03 10:09:56] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.00it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  3.99it/s]

[2025-08-03 10:09:56] Update weights end.
[2025-08-03 10:09:56] Cache flushed successfully!
[2025-08-03 10:09:56] INFO:     127.0.0.1:42124 - "POST /update_weights_from_disk HTTP/1.1" 200 OK


In [9]:
# failed update with different parameter size or wrong name

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "qwen/qwen2.5-0.5b-instruct-wrong"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(response_json)
assert response_json["success"] is False
assert response_json["message"] == (
    "Failed to get weights iterator: "
    "qwen/qwen2.5-0.5b-instruct-wrong"
    " (repository not found)."
)

[2025-08-03 10:09:56] Start update_weights. Load format=auto
[2025-08-03 10:09:56] Update engine weights online from disk begin. avail mem=41.46 GB
[2025-08-03 10:09:56] Failed to get weights iterator: qwen/qwen2.5-0.5b-instruct-wrong (repository not found).
[2025-08-03 10:09:56] INFO:     127.0.0.1:42138 - "POST /update_weights_from_disk HTTP/1.1" 400 Bad Request


In [10]:
terminate_process(server_process)

## Encode (embedding model)

Encode text into embeddings. Note that this API is only available for [embedding models](openai_api_embeddings.html#openai-apis-embedding) and will raise an error for generation models.
Therefore, we launch a new server to server an embedding model.

In [11]:
embedding_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path Alibaba-NLP/gte-Qwen2-1.5B-instruct \
    --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-08-03 10:10:03] server_args=ServerArgs(model_path='Alibaba-NLP/gte-Qwen2-1.5B-instruct', tokenizer_path='Alibaba-NLP/gte-Qwen2-1.5B-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=True, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=32962, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.874, max_running_requests=200, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_s

[2025-08-03 10:10:04] Using default HuggingFace chat template with detected content format: string


[2025-08-03 10:10:11] Downcasting torch.float32 to torch.float16.


[2025-08-03 10:10:12] Overlap scheduler is disabled for embedding models.
[2025-08-03 10:10:12] Downcasting torch.float32 to torch.float16.
[2025-08-03 10:10:12] Attention backend not explicitly specified. Use fa3 backend by default.
[2025-08-03 10:10:12] Init torch distributed begin.


[2025-08-03 10:10:13] Init torch distributed ends. mem usage=0.00 GB


...

[2025-08-03 10:10:17] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=41.51 GB


[2025-08-03 10:10:18] INFO:     Started server process [3926683]
[2025-08-03 10:10:18] INFO:     Waiting for application startup.
[2025-08-03 10:10:18] INFO:     Application startup complete.
[2025-08-03 10:10:18] INFO:     Uvicorn running on http://0.0.0.0:32962 (Press CTRL+C to quit)


[2025-08-03 10:10:19] INFO:     127.0.0.1:36116 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-03 10:10:19] INFO:     127.0.0.1:36132 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-03 10:10:19] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-03 10:10:20] INFO:     127.0.0.1:36140 - "POST /encode HTTP/1.1" 200 OK
[2025-08-03 10:10:20] The server is fired up and ready to roll!


In [12]:
# successful encode for embedding model

url = f"http://localhost:{port}/encode"
data = {"model": "Alibaba-NLP/gte-Qwen2-1.5B-instruct", "text": "Once upon a time"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(f"Text embedding (first 10): {response_json['embedding'][:10]}")

[2025-08-03 10:10:24] Prefill batch. #new-seq: 1, #new-token: 4, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-03 10:10:24] INFO:     127.0.0.1:36144 - "POST /encode HTTP/1.1" 200 OK


In [13]:
terminate_process(embedding_process)

## v1/rerank (cross encoder rerank model)
Rerank a list of documents given a query using a cross-encoder model. Note that this API is only available for cross encoder model like [BAAI/bge-reranker-v2-m3](https://huggingface.co/BAAI/bge-reranker-v2-m3) with `attention-backend` `triton` and `torch_native`.


In [14]:
reranker_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path BAAI/bge-reranker-v2-m3 \
    --host 0.0.0.0 --disable-radix-cache --chunked-prefill-size -1 --attention-backend triton --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-08-03 10:10:31] server_args=ServerArgs(model_path='BAAI/bge-reranker-v2-m3', tokenizer_path='BAAI/bge-reranker-v2-m3', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=True, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=32256, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.874, max_running_requests=200, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=873028795, constrained

[2025-08-03 10:10:32] No chat template found, defaulting to 'string' content format


[2025-08-03 10:10:39] Downcasting torch.float32 to torch.float16.


[2025-08-03 10:10:39] Overlap scheduler is disabled for embedding models.
[2025-08-03 10:10:39] Downcasting torch.float32 to torch.float16.
[2025-08-03 10:10:39] Init torch distributed begin.


[2025-08-03 10:10:40] Init torch distributed ends. mem usage=0.00 GB


...

[2025-08-03 10:10:43] max_total_num_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, max_running_requests=200, context_len=8194, available_gpu_mem=43.30 GB


[2025-08-03 10:10:44] INFO:     Started server process [3929486]
[2025-08-03 10:10:44] INFO:     Waiting for application startup.
[2025-08-03 10:10:44] INFO:     Application startup complete.
[2025-08-03 10:10:44] INFO:     Uvicorn running on http://0.0.0.0:32256 (Press CTRL+C to quit)


[2025-08-03 10:10:45] INFO:     127.0.0.1:45432 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-03 10:10:45] INFO:     127.0.0.1:45434 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-03 10:10:45] Prefill batch. #new-seq: 1, #new-token: 8, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-03 10:10:46] INFO:     127.0.0.1:45440 - "POST /encode HTTP/1.1" 200 OK
[2025-08-03 10:10:46] The server is fired up and ready to roll!


In [15]:
# compute rerank scores for query and documents

url = f"http://localhost:{port}/v1/rerank"
data = {
    "model": "BAAI/bge-reranker-v2-m3",
    "query": "what is panda?",
    "documents": [
        "hi",
        "The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.",
    ],
}

response = requests.post(url, json=data)
response_json = response.json()
for item in response_json:
    print_highlight(f"Score: {item['score']:.2f} - Document: '{item['document']}'")

[2025-08-03 10:10:50] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-03 10:10:50] Prefill batch. #new-seq: 1, #new-token: 43, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-03 10:10:50] INFO:     127.0.0.1:48426 - "POST /v1/rerank HTTP/1.1" 200 OK


In [16]:
terminate_process(reranker_process)

## Classify (reward model)

SGLang Runtime also supports reward models. Here we use a reward model to classify the quality of pairwise generations.

In [17]:
# Note that SGLang now treats embedding models and reward models as the same type of models.
# This will be updated in the future.

reward_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path Skywork/Skywork-Reward-Llama-3.1-8B-v0.2 --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-08-03 10:10:57] server_args=ServerArgs(model_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=True, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=36538, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.874, max_running_requests=200, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False

[2025-08-03 10:10:57] Using default HuggingFace chat template with detected content format: string


[2025-08-03 10:11:07] Overlap scheduler is disabled for embedding models.
[2025-08-03 10:11:07] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-03 10:11:07] Init torch distributed begin.


[2025-08-03 10:11:07] Init torch distributed ends. mem usage=0.00 GB


[2025-08-03 10:11:08] Load weight begin. avail mem=75.63 GB


...

Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.11it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.69it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.45it/s]

[2025-08-03 10:11:11] Load weight end. type=LlamaForSequenceClassification, dtype=torch.bfloat16, avail mem=60.46 GB, mem usage=15.18 GB.
[2025-08-03 10:11:11] KV Cache is allocated. #tokens: 20480, K size: 1.25 GB, V size: 1.25 GB
[2025-08-03 10:11:11] Memory pool end. avail mem=57.66 GB


[2025-08-03 10:11:12] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=57.09 GB


[2025-08-03 10:11:13] INFO:     Started server process [3932583]
[2025-08-03 10:11:13] INFO:     Waiting for application startup.
[2025-08-03 10:11:13] INFO:     Application startup complete.
[2025-08-03 10:11:13] INFO:     Uvicorn running on http://0.0.0.0:36538 (Press CTRL+C to quit)


[2025-08-03 10:11:13] INFO:     127.0.0.1:48046 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-03 10:11:14] INFO:     127.0.0.1:48050 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-03 10:11:14] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


In [18]:
from transformers import AutoTokenizer

PROMPT = (
    "What is the range of the numeric output of a sigmoid node in a neural network?"
)

RESPONSE1 = "The output of a sigmoid node is bounded between -1 and 1."
RESPONSE2 = "The output of a sigmoid node is bounded between 0 and 1."

CONVS = [
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE1}],
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE2}],
]

tokenizer = AutoTokenizer.from_pretrained("Skywork/Skywork-Reward-Llama-3.1-8B-v0.2")
prompts = tokenizer.apply_chat_template(CONVS, tokenize=False)

url = f"http://localhost:{port}/classify"
data = {"model": "Skywork/Skywork-Reward-Llama-3.1-8B-v0.2", "text": prompts}

responses = requests.post(url, json=data).json()
for response in responses:
    print_highlight(f"reward: {response['embedding'][0]}")

[2025-08-03 10:12:19] Prefill batch. #new-seq: 2, #new-token: 136, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-03 10:12:19] INFO:     127.0.0.1:48058 - "POST /encode HTTP/1.1" 200 OK
[2025-08-03 10:12:19] The server is fired up and ready to roll!


[2025-08-03 10:12:33] INFO:     127.0.0.1:53380 - "POST /classify HTTP/1.1" 200 OK


In [19]:
terminate_process(reward_process)

## Capture expert selection distribution in MoE models

SGLang Runtime supports recording the number of times an expert is selected in a MoE model run for each expert in the model. This is useful when analyzing the throughput of the model and plan for optimization.

*Note: We only print out the first 10 lines of the csv below for better readability. Please adjust accordingly if you want to analyze the results more deeply.*

In [20]:
expert_record_server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path Qwen/Qwen1.5-MoE-A2.7B --host 0.0.0.0 --expert-distribution-recorder-mode stat"
)

wait_for_server(f"http://localhost:{port}")

[2025-08-03 10:12:41] server_args=ServerArgs(model_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=37417, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.874, max_running_requests=200, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=835614402, constraine

[2025-08-03 10:12:42] Using default HuggingFace chat template with detected content format: string


[2025-08-03 10:12:50] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-03 10:12:50] Init torch distributed begin.


[2025-08-03 10:12:52] Init torch distributed ends. mem usage=0.10 GB


[2025-08-03 10:12:53] Load weight begin. avail mem=57.19 GB
[2025-08-03 10:12:54] Using model weights format ['*.safetensors']


...

[2025-08-03 10:13:00] KV Cache is allocated. #tokens: 20480, K size: 1.88 GB, V size: 1.88 GB
[2025-08-03 10:13:00] Memory pool end. avail mem=47.85 GB


[2025-08-03 10:13:00] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=8192, available_gpu_mem=47.28 GB


[2025-08-03 10:13:01] INFO:     Started server process [3940292]
[2025-08-03 10:13:01] INFO:     Waiting for application startup.
[2025-08-03 10:13:01] INFO:     Application startup complete.
[2025-08-03 10:13:01] INFO:     Uvicorn running on http://0.0.0.0:37417 (Press CTRL+C to quit)


[2025-08-03 10:13:02] INFO:     127.0.0.1:41826 - "GET /v1/models HTTP/1.1" 200 OK
[2025-08-03 10:13:02] INFO:     127.0.0.1:41838 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-03 10:13:02] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-03 10:13:02] Using default MoE kernel config. Performance might be sub-optimal! Config file not found at /public_sglang_ci/runner-l1b-gpu-45/_work/sglang/sglang/python/sglang/srt/layers/moe/fused_moe_triton/configs/triton_3_3_1/E=60,N=1408,device_name=NVIDIA_H100_80GB_HBM3.json, you can create them with https://github.com/sgl-project/sglang/tree/main/benchmark/kernels/fused_moe_triton


In [21]:
response = requests.post(f"http://localhost:{port}/start_expert_distribution_record")
print_highlight(response)

url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

response = requests.post(f"http://localhost:{port}/stop_expert_distribution_record")
print_highlight(response)

response = requests.post(f"http://localhost:{port}/dump_expert_distribution_record")
print_highlight(response)

[2025-08-03 10:13:16] Resetting ExpertDistributionRecorder...
[2025-08-03 10:13:16] INFO:     127.0.0.1:42738 - "POST /start_expert_distribution_record HTTP/1.1" 200 OK


[2025-08-03 10:13:16] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 1, #queue-req: 0, 


[2025-08-03 10:13:17] INFO:     127.0.0.1:41840 - "POST /generate HTTP/1.1" 200 OK
[2025-08-03 10:13:17] The server is fired up and ready to roll!


[2025-08-03 10:13:18] Decode batch. #running-req: 1, #token: 46, token usage: 0.00, cuda graph: False, gen throughput (token/s): 2.62, #queue-req: 0, 


[2025-08-03 10:13:19] Decode batch. #running-req: 1, #token: 86, token usage: 0.00, cuda graph: False, gen throughput (token/s): 38.52, #queue-req: 0, 


[2025-08-03 10:13:19] INFO:     127.0.0.1:53294 - "POST /generate HTTP/1.1" 200 OK


[2025-08-03 10:13:19] INFO:     127.0.0.1:53302 - "POST /stop_expert_distribution_record HTTP/1.1" 200 OK


[2025-08-03 10:13:22] Write expert distribution to /tmp/expert_distribution_recorder_1754216002.0232375.pt
[2025-08-03 10:13:22] Resetting ExpertDistributionRecorder...
[2025-08-03 10:13:22] INFO:     127.0.0.1:53308 - "POST /dump_expert_distribution_record HTTP/1.1" 200 OK


In [22]:
terminate_process(expert_record_server_process)